In [5]:
import sys
sys.path.append('../src')
sys.path.append('../models')

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from data_processing import DataProcessor
from autoencoder import Autoencoder
from utils import save_model, load_model, create_timestamped_dir, save_training_state, load_training_state
from config import *

c:\Users\Talinho\OneDrive - Insper - Institudo de Ensino e Pesquisa\Área de Trabalho\Insper\NLP\cookembeddings\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


ModuleNotFoundError: No module named 'models'

In [8]:
sentences[0]

'Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet.\nCombine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.\nAdd bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.\nPlace onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.\nPlace a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, 

In [9]:
import torch.nn.functional as F

def contrastive_loss(embeddings_a, embeddings_b, similarity_target=1.0):
    cosine_sim = F.cosine_similarity(embeddings_a, embeddings_b)
    loss = F.mse_loss(cosine_sim, torch.ones_like(cosine_sim) * similarity_target)
    return loss

In [ ]:
from transformers import AdamW
from tqdm import tqdm
import time

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.train()
for epoch in range(num_epochs):
    # Barra de progresso para cada epoch
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)
    epoch_loss = 0
    for batch_idx, batch in enumerate(progress_bar):
        start_time = time.time()
        inputs_a, inputs_b = batch
        inputs_a = {k: v.squeeze().to(device) for k, v in inputs_a.items()}
        inputs_b = {k: v.squeeze().to(device) for k, v in inputs_b.items()}

        # Obter embeddings
        outputs_a = model(**inputs_a).last_hidden_state.mean(dim=1)
        outputs_b = model(**inputs_b).last_hidden_state.mean(dim=1)

        # Calcular a perda
        loss = contrastive_loss(outputs_a, outputs_b)
        epoch_loss += loss.item()

        # Backpropagation e atualização dos pesos
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Atualizar a barra de progresso com a perda média e o tempo por batch
        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "avg_loss": epoch_loss / (batch_idx + 1),
            "batch_time": f"{time.time() - start_time:.2f}s"
        })

    # Print da perda média por epoch após cada época
    print(f"Epoch {epoch + 1} completed. Average Loss: {epoch_loss / len(train_dataloader):.4f}")

c:\Users\Talinho\OneDrive - Insper - Institudo de Ensino e Pesquisa\Área de Trabalho\Insper\NLP\cookembeddings\.venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 1 completed. Average Loss: 0.0000


Epoch 2 completed. Average Loss: 0.0000


Epoch 3 completed. Average Loss: 0.0000


In [11]:
model.save_pretrained("../models/finetuned-bert")
tokenizer.save_pretrained("../models/finetuned-tokenizer")

('../models/finetuned-tokenizer\\tokenizer_config.json',
 '../models/finetuned-tokenizer\\special_tokens_map.json',
 '../models/finetuned-tokenizer\\vocab.txt',
 '../models/finetuned-tokenizer\\added_tokens.json')

In [8]:
from sentence_transformers import SentenceTransformer, models
import torch

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [9]:
from datasets import load_dataset

dataset_id = "embedding-data/QQP_triplets"
dataset = load_dataset(dataset_id)

In [10]:
from sentence_transformers import InputExample

train_examples = []
train_data = dataset['train']['set']
# We will only train 1/2 of our available data
n_examples = dataset['train'].num_rows // 2
for i in range(n_examples):
  example = train_data[i]
  # Add your converted data into train_examples array
  train_examples.append(InputExample(texts=[example['query'],example['pos'][0],example['neg'][0]]))

  0%|          | 0/12724 [39:48<?, ?it/s]


In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [12]:
from sentence_transformers import losses
train_loss = losses.TripletLoss(model)

In [15]:
# we will put epoch value as 4 which defines the total number of iterations during training
import torch
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4)

  0%|          | 0/12724 [00:00<?, ?it/s]

TypeError: SentenceTransformerTrainer.compute_loss() got an unexpected keyword argument 'num_items_in_batch'